In [ ]:
#To Run jupyter notebook, you need to download xview training dataset and training labels from  https://challenge.xviewdataset.org/download-links
#after creating

In [16]:
import geojson
from datasets import Image as Im
import pyarrow as pa
import pandas as pd
from datasets import Dataset
import datasets
from PIL import Image
import numpy as np

In [17]:
with open('xview/train_labels/xView_train.geojson') as f:
    gj = geojson.load(f)
df = pd.json_normalize([feat for feat in gj['features']])
df = df.reset_index()
del gj

In [18]:
df = df[df['properties.image_id'] != '1395.tif']
df = df.reset_index()
df.drop(['level_0', 'index'], axis=1, inplace=True)

In [19]:
df['properties.image_location'] = 'xview/train_images/' + df['properties.image_id']
df['list_of_cooridates'] = df['properties.bounds_imcoords'].str.split(",")
df['list_of_cooridates'] =  df['list_of_cooridates'].apply(lambda x: [int(el) for el in x])
#df['shape'] = df['properties.image_location'].progress_apply(lambda x: np.array(Image.open(x)).shape)
df['first'] = df['list_of_cooridates'].map(lambda x: int(x[0]))
df['second'] = df['list_of_cooridates'].map(lambda x: int(x[1]))
df['third'] = df['list_of_cooridates'].map(lambda x: int(x[2]))
df['fourth'] = df['list_of_cooridates'].map(lambda x: int(x[3]))
df['area'] = df.apply(lambda x: (x['third'] - x['first']) * (x['fourth'] - x['second']), axis=1)

In [20]:
df['shape'] = 0
df_final = pd.DataFrame()
LIST =[]; i = 0
for group_name, df2 in df.groupby('properties.image_location'):
    shape_val = list(np.array(Image.open(group_name)).shape)

    df2['shape']=[shape_val for x in range(0,df2.shape[0])]
    
    df2['properties.image_location'] = group_name
    LIST.append(df2)
    if len(LIST) > 20:
        df_concat = pd.concat(LIST)
        df_final = pd.concat([df_final,df_concat])
        LIST = []
        print('finished with ' + str(i))
    i += 1
if len(LIST) > 0:
    df_concat = pd.concat(LIST)
    df_final = pd.concat([df_final,df_concat])
    
df = df_final.reset_index()
del df_final, df2,df_concat

finished with 20
finished with 41
finished with 62
finished with 83
finished with 104
finished with 125
finished with 146
finished with 167
finished with 188
finished with 209
finished with 230
finished with 251
finished with 272
finished with 293
finished with 314
finished with 335
finished with 356
finished with 377
finished with 398
finished with 419
finished with 440
finished with 461
finished with 482
finished with 503
finished with 524
finished with 545
finished with 566
finished with 587
finished with 608
finished with 629
finished with 650
finished with 671
finished with 692
finished with 713
finished with 734
finished with 755
finished with 776
finished with 797
finished with 818
finished with 839


In [21]:
df['shape_1'] = df['shape'].map(lambda x: int(x[0]))
df['shape_2'] = df['shape'].map(lambda x: int(x[1]))

In [22]:
df['third']=df[['third','shape_2']].min(axis=1)
df['fourth']=df[['fourth','shape_1']].min(axis=1)
df['first']=df['first'].clip(0, None)
df['second']=df['second'].clip(0, None)

In [23]:
df = df.drop(df[((df['first'] == df['third']) | (df['second'] == df['fourth']) )].index)
df = df.reset_index()

In [24]:
print('check to make sure crop doesn\'t take place outside image')
print(df[((df.third > df['shape_2']) | (df.fourth > df['shape_1']) )][['list_of_cooridates', 'shape']])
print(df[((df['first'] > df['third']) | (df['second'] > df['fourth']) )][['list_of_cooridates', 'shape']])
print(df[((df['first'] < 0) | (df['second'] < 0) )][['list_of_cooridates', 'shape']])
print(df[((df['first'] == df['third']) | (df['second'] == df['fourth']) )][['list_of_cooridates', 'shape']])

check to make sure crop doesn't take place outside image
Empty DataFrame
Columns: [list_of_cooridates, shape]
Index: []
Empty DataFrame
Columns: [list_of_cooridates, shape]
Index: []
Empty DataFrame
Columns: [list_of_cooridates, shape]
Index: []
Empty DataFrame
Columns: [list_of_cooridates, shape]
Index: []


In [36]:
df['list_of_cooridates'] = df.apply(lambda x: [int(x['first']),int(x['second']),int(x['third']),int(x['fourth'])],axis=1)
df['first'] = df['list_of_cooridates'].map(lambda x: int(x[0]))
df['second'] = df['list_of_cooridates'].map(lambda x: int(x[1]))
df['third'] = df['list_of_cooridates'].map(lambda x: int(x[2]))
df['fourth'] = df['list_of_cooridates'].map(lambda x: int(x[3]))
df['area'] = df.apply(lambda x: (x['third'] - x['first']) * (x['fourth'] - x['second']), axis=1)

In [37]:
print('check to make sure crop doesn\'t take place outside image')
print(df[((df.third > df['shape_2']) | (df.fourth > df['shape_1']) )][['list_of_cooridates', 'shape']])
print(df[((df['first'] > df['third']) | (df['second'] > df['fourth']) )][['list_of_cooridates', 'shape']])
print(df[((df['first'] < 0) | (df['second'] < 0) )][['list_of_cooridates', 'shape']])
print(df[((df['first'] == df['third']) | (df['second'] == df['fourth']) )][['list_of_cooridates', 'shape']])

check to make sure crop doesn't take place outside image
Empty DataFrame
Columns: [list_of_cooridates, shape]
Index: []
Empty DataFrame
Columns: [list_of_cooridates, shape]
Index: []
Empty DataFrame
Columns: [list_of_cooridates, shape]
Index: []
Empty DataFrame
Columns: [list_of_cooridates, shape]
Index: []


In [38]:
df['shape'] = 0
df_final = pd.DataFrame()
LIST =[]; i = 0
for group_name, df2 in df.groupby('properties.image_location'):
    df2 = df2.drop(['level_0'],axis=1)
    df2 = df2.reset_index()
    df_append = pd.DataFrame(index=range(1),columns=['image_id','image','width','height','objects'])
    
    height, width, _ = np.array(Image.open(group_name)).shape
    df_append.at[0,'image_id'] = df2['properties.image_id'][0]
    df_append.at[0,'image'] = group_name
    df_append.at[0,'width'] = width
    df_append.at[0,'height'] = height
    df_append.at[0,'objects'] = dict({
        'id': df2['index'].tolist(),
        'area': df2['area'].tolist(),
        'bbox': df2['list_of_cooridates'].tolist(),
        'category': df2['properties.type_id'].tolist()
    }
    )
    
    LIST.append(df_append)
    if len(LIST) > 20:
        df_concat = pd.concat(LIST)
        df_final = pd.concat([df_final,df_concat])
        LIST = []
        print('finished with ' + str(i))
    i += 1
if len(LIST) > 0:
    df_concat = pd.concat(LIST)
    df_final = pd.concat([df_final,df_concat])
    
df = df_final.reset_index()
del df_final, df2,df_concat, df_append

finished with 20
finished with 41
finished with 62
finished with 83
finished with 104
finished with 125
finished with 146
finished with 167
finished with 188
finished with 209
finished with 230
finished with 251
finished with 272
finished with 293
finished with 314
finished with 335
finished with 356
finished with 377
finished with 398
finished with 419
finished with 440
finished with 461
finished with 482
finished with 503
finished with 524
finished with 545
finished with 566
finished with 587
finished with 608
finished with 629
finished with 650
finished with 671
finished with 692
finished with 713
finished with 734
finished with 755
finished with 776
finished with 797
finished with 818
finished with 839


In [39]:
hg_dataset = Dataset(pa.Table.from_pandas(df))
NewDataset = datasets.DatasetDict({"train":hg_dataset})#,"val":test_dataset})
newdata = NewDataset['train'].cast_column("image", Im())
NewDataset = datasets.DatasetDict({"train":newdata})#,"val":test_dataset['train']})
del newdata, hg_dataset

In [40]:
NewDataset['train'].data.nbytes / 1e9

0.036179418

In [41]:
NewDataset


DatasetDict({
    train: Dataset({
        features: ['index', 'image_id', 'image', 'width', 'height', 'objects'],
        num_rows: 846
    })
})

In [42]:
NewDataset['train'][0]


{'index': 0,
 'image_id': '10.tif',
 'image': <PIL.TiffImagePlugin.TiffImageFile image mode=RGB size=3320x3228>,
 'width': 3320,
 'height': 3228,
 'objects': {'area': [231,
   392,
   5494,
   3596,
   532,
   1815,
   1806,
   1656,
   608,
   520,
   682,
   640,
   1829,
   4836,
   5609,
   1419,
   2805,
   2478,
   1435,
   1326,
   1054,
   1728,
   1292,
   868,
   930,
   2166,
   702,
   7885,
   1334,
   759,
   2548,
   1681,
   3154,
   2584,
   450,
   624,
   247,
   104,
   625,
   3306,
   2400,
   1564,
   320,
   5376,
   696,
   1650,
   5698,
   468,
   630,
   784,
   3720,
   1224,
   704,
   3905,
   3283,
   2209,
   1845,
   520,
   3478,
   4260,
   2464,
   361,
   5175],
  'bbox': [[1409, 732, 1430, 743],
   [1188, 466, 1216, 480],
   [1639, 37, 1706, 119],
   [1558, 416, 1620, 474],
   [1594, 581, 1622, 600],
   [1576, 591, 1631, 624],
   [1805, 635, 1847, 678],
   [1773, 634, 1809, 680],
   [1458, 706, 1490, 725],
   [1403, 709, 1429, 729],
   [1490, 542,

In [43]:
NewDataset.push_to_hub('Honaker/xview_dataset',  num_shards={'train': 50, 'test': 0})

Pushing dataset shards to the dataset hub: 100%|██████████| 50/50 [4:20:58<00:00, 313.16s/it]
